In [7]:
import librosa
import numpy as np

In [8]:
root_audio = "/home/data/kbh/DCASE2022_SELD_synth_data/foa/"
target = "fold1_room10_mix001"

# ms
shift = 256
n_fft = 1024
fs = 24000
dim_feature = 64

In [9]:
raw,_ = librosa.load(root_audio+target+".wav",sr=fs,mono=False)
raw = raw[:,:72000]
print("raw : {}".format(raw.shape))

raw : (4, 72000)


In [10]:
def get_foa_intensity_vectors(linear_spectra,n_mels,eps=1e-8):
    mel_filter = librosa.filters.mel(sr=fs, n_fft=n_fft, n_mels=n_mels).T
    W = linear_spectra[:, :, 0]
    I = np.real(np.conj(W)[:, :, np.newaxis] * linear_spectra[:, :, 1:])
    E = eps + (np.abs(W)**2 + ((np.abs(linear_spectra[:, :, 1:])**2).sum(-1))/3.0 )
    
    I_norm = I/E[:, :, np.newaxis]
    I_norm_mel = np.transpose(np.dot(np.transpose(I_norm, (0,2,1)), mel_filter), (1,0,2))
    """
    foa_iv = I_norm_mel.transpose((0, 2, 1)).reshape((linear_spectra.shape[0], n_mels * 3))
    if np.isnan(foa_iv).any():
        print('Feature extraction is generating nan outputs')
        exit()
    """
    return I_norm_mel

In [11]:
spec = []
for c in range(4) : 
    spec.append(librosa.stft(raw[c,:],n_fft=n_fft))
spec = np.array(spec)
print(spec.shape)

(4, 513, 282)


In [27]:
IV = get_foa_intensity_vectors(spec.T, n_mels = dim_feature)
print(IV.shape)


mfcc = librosa.feature.mfcc(raw[c,:], sr=fs, n_mfcc=dim_feature, n_fft = n_fft,hop_length=shift)
print(mfcc.shape)

# mono 만 가능
chroma_stft = librosa.feature.chroma_stft(raw[0], sr=fs, n_fft = n_fft,hop_length=shift)
print(chroma_stft.shape)

# 아래는 mag만 가능
spectral_centroid = librosa.feature.spectral_centroid(S=np.abs(spec)[0], sr=fs, n_fft = n_fft,hop_length=shift)
print(spectral_centroid.shape)

spectral_bandwidth = librosa.feature.spectral_bandwidth(S=np.abs(spec)[0], sr=fs, n_fft = n_fft,hop_length=shift)
print(spectral_bandwidth.shape)

spectral_contrast = librosa.feature.spectral_contrast(S=np.abs(spec)[0], sr=fs, n_fft = n_fft,hop_length=shift)
print(spectral_contrast.shape)

spectral_flatness = librosa.feature.spectral_flatness(S=np.abs(spec)[0], n_fft = n_fft,hop_length=shift)
print(spectral_flatness.shape)

(3, 282, 64)
(64, 282)
(12, 282)
(1, 282)
(1, 282)
(7, 282)
(1, 282)


In [34]:
print(chroma_stft)

[[1.         0.91481304 0.95095545 ... 0.04832633 0.11612966 0.20021468]
 [0.9393179  0.9313058  0.89147884 ... 0.08728583 0.12523416 0.09456823]
 [0.9132628  0.91995597 0.86078674 ... 0.4045147  0.25959364 0.06088699]
 ...
 [0.7191333  0.7757538  0.98636883 ... 0.38417563 0.7653442  0.9821886 ]
 [0.74931765 0.7490644  0.9968407  ... 0.2569745  0.52122104 0.87429273]
 [0.8444935  0.7614243  0.9606041  ... 0.11358879 0.25830454 0.46830082]]


In [46]:
spec[0].shape

(513, 282)

In [2]:
import torch
data = torch.load("/home/data/kbh/DCASE2022/d0/test/fold4_room10_mix001_0_.pt")

In [6]:
data["data"].shape

(4, 282, 126)

In [35]:
import torch

In [43]:
x = torch.tensor((1,2,3))
y= torch.zeros((2,3))
y[0,:]=x

In [44]:
y

tensor([[1., 2., 3.],
        [0., 0., 0.]])